# Covid19 Analysis for Recovery Phase Prediction

Source Datasets:

1. Our World in Data COVID-19 Testing dataset https://ourworldindata.org/coronavirus-testing
1. Google Covid19 Community Mobility Reports https://www.google.com/covid19/mobility/

Methdology:

1. Aggregate source data at daily country level and consolidate as master data
1. Calculate risk index based on rate of infection and total number of infections
1. Train machine learning model to translate relationship between risk drivers (mobility and/or restriction) and risk index

Prediction Scenario:
1. To prevent second spike in recovery phase, one would be able to predict on the risk index given the assumed risk drivers therefore to adjust government policies and country wide communications

In [3]:
#load coviddata dataset
import pandas as pd
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
coviddata=pd.read_csv(url)
#coviddata = coviddata[coviddata['date']>='2020-03-01']

#change data types and column names
coviddata = coviddata[coviddata['iso_code']!='OWID_WRL']
coviddata['date'] = pd.to_datetime(coviddata['date'])
coviddata.rename(columns={'iso_code':'iso3_code'}, inplace=True)

display(coviddata)

iso3_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
AFG,Asia,Afghanistan,2019-12-31T00:00:00.000+0000,0,0,0,0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,3.8928341E7,54.422,18.6,2.5810000000000004,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5
AFG,Asia,Afghanistan,2020-01-01T00:00:00.000+0000,0,0,0,0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,3.8928341E7,54.422,18.6,2.5810000000000004,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5
AFG,Asia,Afghanistan,2020-01-02T00:00:00.000+0000,0,0,0,0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,3.8928341E7,54.422,18.6,2.5810000000000004,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5
AFG,Asia,Afghanistan,2020-01-03T00:00:00.000+0000,0,0,0,0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,3.8928341E7,54.422,18.6,2.5810000000000004,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5
AFG,Asia,Afghanistan,2020-01-04T00:00:00.000+0000,0,0,0,0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,3.8928341E7,54.422,18.6,2.5810000000000004,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5
AFG,Asia,Afghanistan,2020-01-05T00:00:00.000+0000,0,0,0,0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,3.8928341E7,54.422,18.6,2.5810000000000004,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5
AFG,Asia,Afghanistan,2020-01-06T00:00:00.000+0000,0,0,0,0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,3.8928341E7,54.422,18.6,2.5810000000000004,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5
AFG,Asia,Afghanistan,2020-01-07T00:00:00.000+0000,0,0,0,0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,3.8928341E7,54.422,18.6,2.5810000000000004,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5
AFG,Asia,Afghanistan,2020-01-08T00:00:00.000+0000,0,0,0,0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,3.8928341E7,54.422,18.6,2.5810000000000004,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5
AFG,Asia,Afghanistan,2020-01-09T00:00:00.000+0000,0,0,0,0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,3.8928341E7,54.422,18.6,2.5810000000000004,1.337,1803.987,null,597.029,9.59,null,null,37.746,0.5


In [4]:
import country_converter as coco

iso2_code = coco.convert(names=coviddata['iso3_code'].to_list(), to='ISO2', not_found=None)

coviddata['iso2_code'] = iso2_code
coviddata = coviddata[coviddata['iso2_code']!='None']

WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:OWID_KOS not found in regex
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:nan not found in ISO3
WARNING:root:

In [5]:
#load mobility dataset
url="https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=6d352e35dcffafce"
mobility=pd.read_csv(url)

#mobility = mobility[mobility['date']>='2020-03-01']
display(mobility)

country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
AE,United Arab Emirates,null,null,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
AE,United Arab Emirates,null,null,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
AE,United Arab Emirates,null,null,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
AE,United Arab Emirates,null,null,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
AE,United Arab Emirates,null,null,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
AE,United Arab Emirates,null,null,2020-02-20,-2.0,1.0,6.0,1.0,1.0,1.0
AE,United Arab Emirates,null,null,2020-02-21,-3.0,2.0,6.0,0.0,-1.0,1.0
AE,United Arab Emirates,null,null,2020-02-22,-2.0,2.0,4.0,-2.0,3.0,1.0
AE,United Arab Emirates,null,null,2020-02-23,-1.0,3.0,3.0,-1.0,4.0,1.0
AE,United Arab Emirates,null,null,2020-02-24,-3.0,0.0,5.0,-1.0,3.0,1.0


In [6]:
#change data type
mobility['date']= pd.to_datetime(mobility['date'])

mobility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508874 entries, 0 to 508873
Data columns (total 11 columns):
country_region_code 507996 non-null object
country_region 508874 non-null object
sub_region_1 493754 non-null object
sub_region_2 294499 non-null object
date 508874 non-null datetime64[ns]
retail_and_recreation_percent_change_from_baseline 394195 non-null float64
grocery_and_pharmacy_percent_change_from_baseline 379456 non-null float64
parks_percent_change_from_baseline 234590 non-null float64
transit_stations_percent_change_from_baseline 270823 non-null float64
workplaces_percent_change_from_baseline 495623 non-null float64
residential_percent_change_from_baseline 280410 non-null float64
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 42.7+ MB

In [7]:
#aggregate values by date and country
mobility_agg = mobility.groupby(['date','country_region','country_region_code']).agg(
  {'retail_and_recreation_percent_change_from_baseline':'sum',
   'grocery_and_pharmacy_percent_change_from_baseline':'sum',
   'parks_percent_change_from_baseline':'sum',
   'transit_stations_percent_change_from_baseline':'sum',
   'workplaces_percent_change_from_baseline':'sum',
   'residential_percent_change_from_baseline':'sum'}).reset_index()

mobility_agg.head()

,date,country_region,country_region_code,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-02-15,Afghanistan,AF,-9.0,-10.0,0.0,-2.0,-28.0,5.0
1,2020-02-15,Angola,AO,3.0,-4.0,10.0,3.0,6.0,2.0
2,2020-02-15,Antigua and Barbuda,AG,-11.0,-1.0,-10.0,4.0,-1.0,3.0
3,2020-02-15,Argentina,AR,2.0,-53.0,-158.0,-134.0,-23.0,32.0
4,2020-02-15,Aruba,AW,2.0,3.0,12.0,26.0,-2.0,-2.0


In [8]:
#join three dataset by country code

#version1 : use datasets (kaggle covid19, population and mobility)
#before_master = covid19_agg.merge(mobility_agg, left_on=['ObservationDate','CountryCode'], right_on=['date','country_region_code'], how='left')
#master = before_master.merge(population, left_on='country_region_code', right_on='Code', how='left')
#master = master.drop(['ObservationDate', 'Country/Region','CountryCode', 'Index', 'Country', 'Country code', 'Code'], axis=1)
#master = master.dropna(, inplace=True)

#version2： use datasets (owid world coviddata with population included and mobility)
master = coviddata.merge(mobility_agg, left_on=['date','iso2_code'], right_on=['date','country_region_code'], how='left')
master = master.dropna(subset=['country_region_code']) 
master = master.drop(['iso3_code', 'iso2_code'], axis=1)

master

,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,country_region,country_region_code,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
46,Asia,Afghanistan,2020-02-15,0,0,0,0,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,Afghanistan,AF,-9.0,-10.0,0.0,-2.0,-28.0,5.0
47,Asia,Afghanistan,2020-02-16,0,0,0,0,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,Afghanistan,AF,3.0,11.0,1.0,10.0,4.0,0.0
48,Asia,Afghanistan,2020-02-17,0,0,0,0,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,Afghanistan,AF,6.0,11.0,2.0,9.0,5.0,-1.0
49,Asia,Afghanistan,2020-02-18,0,0,0,0,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,Afghanistan,AF,3.0,5.0,3.0,9.0,6.0,-1.0
50,Asia,Afghanistan,2020-02-19,0,0,0,0,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,Afghanistan,AF,-1.0,3.0,1.0,0.0,5.0,1.0
51,Asia,Afghanistan,2020-02-20,0,0,0,0,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,Afghanistan,AF,-2.0,3.0,-1.0,4.0,6.0,1.0
52,Asia,Afghanistan,2020-02-21,0,0,0,0,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,Afghanistan,AF,0.0,9.0,4.0,7.0,6.0,0.0
53,Asia,Afghanistan,2020-02-22,0,0,0,0,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,Afghanistan,AF,2.0,5.0,6.0,7.0,6.0,0.0
54,Asia,Afghanistan,2020-02-23,0,0,0,0,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,Afghanistan,AF,2.0,6.0,2.0,7.0,6.0,1.0
55,Asia,Afghanistan,2020-02-24,0,0,0,0,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,Afghanistan,AF,3.0,13.0,4.0,9.0,7.0,0.0


In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#plot on residential mobility stats with selected countries in May
plt.figure(figsize=(20,4))
countrys=['GB','US','JP','FR','CA','DE','IN']
master_selection=master[(master['date']>='2020-05-01') & (master['country_region_code'].isin(countrys))]
ax = sns.lineplot(x="date", y="residential_percent_change_from_baseline", hue="country_region_code", data=master_selection)
plt.yscale('log')
display(plt.show())

Risk Index Formula: 

1. rate of infection: the increase/decrease rate of current total cases against total cases from 14 days ago (normal virus incubation period)
1. number of infection: current total cases per million

Formula refers to uk government risk index guidance: 
https://www.spectator.co.uk/article/how-number-10-should-illustrate-its-covid-alert-formula

In [11]:
#align data with total cases minus 14 days

#version1: total manual weightage assignment
#master['risk_index'] = (master['Confirmed']/master['Year_2020'])*0.2+(master['Deaths']/master['Year_2020'])*0.6-(master['Recovered']/master['Year_2020'])*0.2

#version2: combined use of spead rate r and number of accumulative infections i
master['country_level_index'] = master.groupby(['country_region_code']).cumcount()+1
 
back_track = []
total_cases_minus14 = []
for country in master['country_region_code'].unique():
  for index in range(0,max(master[master['country_region_code']==country]['country_level_index'])):
      try:
        back_track.append(master.loc[(master['country_region_code']==country) & (master['country_level_index']==(index-14)),'total_cases'].values)
      except KeyError:
        back_track.append(float("nan"))
        
import numpy as np
out = []
for i in range(0,len(back_track)):
  if len(back_track[i])==0:
    out.append(np.array([0]))
  else:
    out.append(back_track[i])
  
from itertools import chain
total_cases_minus14 = list(chain(*out))

master['total_cases_minus14'] = total_cases_minus14

In [12]:
#calcualte the two factors that contribute to risk index
master['rate_of_infection'] = (master['total_cases']-master['total_cases_minus14'])/master['total_cases_minus14']
master['number_of_infection'] = master['total_cases_per_million']

#plot on matrix of rate_of_infection and number_of_infection with a selection of countries
plt.figure(figsize=(6,6))
countrys=['GB','US','JP','FR','CA','DE','IN']
master_selection=master[(master['date']>='2020-03-01') & (master['country_region_code'].isin(countrys))]
ax = sns.scatterplot(x="number_of_infection", y="rate_of_infection", hue="country_region_code", data=master_selection)
display(plt.show())

In [13]:
#calcualte risk index 
import math
risk_index = []

for i in range(0,len(master)):
  if math.isinf(master.iloc[i]['rate_of_infection']) or (master.iloc[i]['rate_of_infection']<20 and master.iloc[i]['number_of_infection']<1000):
    risk_index.append(1)
  elif master.iloc[i]['rate_of_infection']<40 and master.iloc[i]['number_of_infection']<2000:
    risk_index.append(2)
  elif master.iloc[i]['rate_of_infection']<60 and master.iloc[i]['number_of_infection']<3000:
    risk_index.append(3)
  elif master.iloc[i]['rate_of_infection']<80 and master.iloc[i]['number_of_infection']<4000:
    risk_index.append(4)
  else:
    risk_index.append(5)

master['risk_index'] = risk_index

#master[master['location']=='United Kingdom']

In [14]:
#output dataset master
df = spark.createDataFrame(master)
#df.write.format("csv").saveAsTable("covid_analysis_with_risk_index")
df.write.mode("overwrite").saveAsTable("covid_analysis_with_risk_index")

In [15]:
dbutils.fs.rm("/FileStore/analysis-output/covid_analysis_with_risk_index.csv",True)
df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/analysis-output/covid_analysis_with_risk_index.csv")
#IMPORTANT: file downloadable at:
#https://adb-5999897127967121.1.azuredatabricks.net/files/analysis-output/covid_analysis_with_risk_index.csv/part-00000-tid-5579663262833822991-197e2b9c-ad1e-474d-a172-faa190badc8f-53-1-c000.csv?o=5999897127967121

Prediction Model Build: 

Make use of a classification model to capture relationship between risk drivers (mobility and/or restriction) and risk index to future prediction that aids recovery phase planning

In [17]:
#split data into a labels dataframe and a features dataframe
labels = master['risk_index'].values
featureNames = ['retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline',
                'parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline',
                'workplaces_percent_change_from_baseline','residential_percent_change_from_baseline']
features = master[featureNames].values

In [18]:
#normalize features (columns) to have unit variance
from sklearn.preprocessing import normalize
features = normalize(features, axis=0)
features

Out[15]: array([[-1.80309050e-05, -4.74387926e-05, 0.00000000e+00,
 -7.61516490e-06, -3.48612742e-05, 3.09600745e-05],
 [ 6.01030166e-06, 5.21826719e-05, 4.86089687e-06,
 3.80758245e-05, 4.98018204e-06, 0.00000000e+00],
 [ 1.20206033e-05, 5.21826719e-05, 9.72179374e-06,
 3.42682421e-05, 6.22522754e-06, -6.19201489e-06],
 ...,
 [-1.52260975e-04, -1.56548016e-04, -2.38183947e-04,
 -5.55907038e-04, -1.76796462e-04, 4.45825072e-04],
 [-1.30223203e-04, -1.09109223e-04, -2.18740359e-04,
 -4.87370554e-04, -1.66836098e-04, 4.45825072e-04],
 [-1.66285013e-04, -1.32828619e-04, -2.43044844e-04,
 -5.17831213e-04, -1.10809050e-04, 2.97216715e-04]])

In [19]:
#hold out 20% of the data for testing with stratification enabled
from sklearn.model_selection import train_test_split
trainingLabels, testLabels, trainingFeatures, testFeatures = train_test_split(labels, features, test_size=0.2, stratify=labels)
ntrain, ntest = len(trainingLabels), len(testLabels)
print('Split data randomly into 2 sets: %d training and %d test instances.' % (ntrain, ntest))

Split data randomly into 2 sets: 10113 training and 2529 test instances.

In [20]:
#train a gradientboosting model with fixed hyperparameters
from sklearn.ensemble import GradientBoostingClassifier
Clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                 max_depth=2, random_state=0)
Clf.fit(features, labels)
#print('Trained model with fixed random_state = 0')

Out[17]: GradientBoostingClassifier(criterion='friedman_mse', init=None,
 learning_rate=1.0, loss='deviance', max_depth=2,
 max_features=None, max_leaf_nodes=None,
 min_impurity_decrease=0.0, min_impurity_split=None,
 min_samples_leaf=1, min_samples_split=2,
 min_weight_fraction_leaf=0.0, n_estimators=100,
 n_iter_no_change=None, presort='auto', random_state=0,
 subsample=1.0, tol=0.0001, validation_fraction=0.1,
 verbose=0, warm_start=False)

In [21]:
#score the model
TrainingScore, TestScore = Clf.score(trainingFeatures, trainingLabels), Clf.score(testFeatures, testLabels)

print('Model  \t\tTraining \tTest')
print('Version 1.0\t%g\t%g' % (TrainingScore, TestScore))

Model Training Test
Version 1.0	0.822802	0.825227

In [22]:
#compare actual vs predicted values
predLabels = Clf.predict(testFeatures)

comparison= pd.DataFrame({'Actual': testLabels.flatten(), 'Predicted': predLabels.flatten()})
#comparison

In [23]:
#plot first 100 comparison results
comparison.head(100).plot(kind='bar',figsize=(20,6))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
display(plt.show())